In [45]:
from google.cloud import documentai_v1 as documentai
from google.api_core.client_options import ClientOptions
from google.cloud import storage
import pandas as pd
import time
from pathlib import Path
import json

# === CONFIG ===
project_id = "vercillopersonal"
location = "us"
processor_id = "fe61eee8945a8018"

# === INPUT/OUTPUT PATHS ===
# gcs_input_uri = "gs://vercillo_projects/transactions/amex/data/2025/2025-01-03_cleansed.csv"
# filename = Path(gcs_input_uri).name  
# gcs_output_uri = "gs://vercillo_projects/transactions/amex/exports/"




## Google Cloud Storage Bucket

In [46]:
# === INPUT/OUTPUT PATHS ===
# gcs_input_uri = "gs://vercillo_projects/transactions/amex/data/2025/2025-01-03_cleansed.csv"
# filename = Path(gcs_input_uri).name  
# gcs_output_uri = "gs://vercillo_projects/transactions/amex/exports/"

# df = pd.read_csv(gcs_input_uri)

## Local File Path

In [52]:
filename = r"C:\Users\jverc\OneDrive\02.DataScienceOD\test_files\2025\2025-01-03_cleansed.csv"
df = pd.read_csv(filename)

In [53]:
df

,row_id,Vendor,amount,location,posting_date,transaction_date,closing_date,opening_date
0,0,PTZ INSURANCE SERVICES,21.27,OAKVILLE,04-Dec,03-Dec,03-Jan-25,04-Dec-24
1,1,SECURITY NATIONAL INSUR,88.18,MONTREAL,05-Dec,04-Dec,03-Jan-25,04-Dec-24
2,2,UBER EATS,20.14,HTTPS://HELP.UB,05-Dec,05-Dec,03-Jan-25,04-Dec-24
3,3,UBER EATS,1,HTTPS://HELP.UB,05-Dec,05-Dec,03-Jan-25,04-Dec-24
4,4,DC03 A-OK COMMISSARY &,5.65,Vaughan,05-Dec,05-Dec,03-Jan-25,04-Dec-24
...,...,...,...,...,...,...,...,...
62,62,points_redeemed,"26,166",NaN,NaN,NaN,03-Jan-25,04-Dec-24
63,63,PAYMENT RECEIVED,-2481.67,NaN,04-Dec,04-Dec,03-Jan-25,04-Dec-24
64,64,Use Points for Purchases | Reference S00024339...,-261.66,NaN,04-Dec,04-Dec,03-Jan-25,04-Dec-24
65,65,PAYMENT RECEIVED,-1000,NaN,22-Dec,22-Dec,03-Jan-25,04-Dec-24


In [54]:
date_cols = ["closing_date", "opening_date"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%d-%b-%y", errors="coerce")

# Extract the year
df["opening_year"] = df["opening_date"].dt.year
df

,row_id,Vendor,amount,location,posting_date,transaction_date,closing_date,opening_date,opening_year
0,0,PTZ INSURANCE SERVICES,21.27,OAKVILLE,04-Dec,03-Dec,2025-01-03,2024-12-04,2024
1,1,SECURITY NATIONAL INSUR,88.18,MONTREAL,05-Dec,04-Dec,2025-01-03,2024-12-04,2024
2,2,UBER EATS,20.14,HTTPS://HELP.UB,05-Dec,05-Dec,2025-01-03,2024-12-04,2024
3,3,UBER EATS,1,HTTPS://HELP.UB,05-Dec,05-Dec,2025-01-03,2024-12-04,2024
4,4,DC03 A-OK COMMISSARY &,5.65,Vaughan,05-Dec,05-Dec,2025-01-03,2024-12-04,2024
...,...,...,...,...,...,...,...,...,...
62,62,points_redeemed,"26,166",NaN,NaN,NaN,2025-01-03,2024-12-04,2024
63,63,PAYMENT RECEIVED,-2481.67,NaN,04-Dec,04-Dec,2025-01-03,2024-12-04,2024
64,64,Use Points for Purchases | Reference S00024339...,-261.66,NaN,04-Dec,04-Dec,2025-01-03,2024-12-04,2024
65,65,PAYMENT RECEIVED,-1000,NaN,22-Dec,22-Dec,2025-01-03,2024-12-04,2024


In [55]:
df["posting_date_full"] = pd.to_datetime(
    df["posting_date"].astype(str) + "-" + df["opening_year"].astype(str),
    format="%d-%b-%Y",
    errors="coerce"
)

df["transaction_date_full"] = pd.to_datetime(
    df["transaction_date"].astype(str) + "-" + df["opening_year"].astype(str),
    format="%d-%b-%Y",
    errors="coerce"
)
df = df.drop(['posting_date', 'transaction_date'], axis=1)

df = df.rename(columns={
    "transaction_date_full": "transaction_date",
    "posting_date_full": "posting_date"
})


df = df[["row_id", "Vendor", "amount", "location","posting_date", "transaction_date","opening_date", "closing_date", "opening_year"]]

In [56]:
df

,row_id,Vendor,amount,location,posting_date,transaction_date,opening_date,closing_date,opening_year
0,0,PTZ INSURANCE SERVICES,21.27,OAKVILLE,2024-12-04,2024-12-03,2024-12-04,2025-01-03,2024
1,1,SECURITY NATIONAL INSUR,88.18,MONTREAL,2024-12-05,2024-12-04,2024-12-04,2025-01-03,2024
2,2,UBER EATS,20.14,HTTPS://HELP.UB,2024-12-05,2024-12-05,2024-12-04,2025-01-03,2024
3,3,UBER EATS,1,HTTPS://HELP.UB,2024-12-05,2024-12-05,2024-12-04,2025-01-03,2024
4,4,DC03 A-OK COMMISSARY &,5.65,Vaughan,2024-12-05,2024-12-05,2024-12-04,2025-01-03,2024
...,...,...,...,...,...,...,...,...,...
62,62,points_redeemed,"26,166",NaN,NaT,NaT,2024-12-04,2025-01-03,2024
63,63,PAYMENT RECEIVED,-2481.67,NaN,2024-12-04,2024-12-04,2024-12-04,2025-01-03,2024
64,64,Use Points for Purchases | Reference S00024339...,-261.66,NaN,2024-12-04,2024-12-04,2024-12-04,2025-01-03,2024
65,65,PAYMENT RECEIVED,-1000,NaN,2024-12-22,2024-12-22,2024-12-04,2025-01-03,2024
